**Câu 1)**

**People You Might Know**

This a Spark program that implements a simple “People You Might Know” social network friendship recommendation algorithm. The key idea is that if two people have a lot of mutual friends, then the system should recommend that they connect with each other.

We use the resilient distributed dataset (RDD).

**Algorithm:**

Let us use a simple algorithm such that, for each user U, the algorithm rec- ommends N = 10 users who are not already friends with U, but have the most number of mutual friends in common with U.

**Requirement:**

PySpark 3.1.2

In [8]:
%%capture
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Avoids scroll-in-the-scroll in the entire Notebook
from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 400})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
%matplotlib inline
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
# create the session
conf = SparkConf().set("spark.ui.port", "4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

<IPython.core.display.Javascript object>

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

<IPython.core.display.Javascript object>

--2023-03-30 13:28:17--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  2.67MB/s    in 12s     

2023-03-30 13:28:30 (1.08 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [ ]:
def line_to_friend_ownership(line):
    split = line.split()
    user_id = int(split[0])
    if len(split) == 1:
        friends = []
    else:
        friends = list(map(lambda x: int(x), split[1].split(',')))
    return user_id, friends

def friend_ownership_to_connection(f_o):
    user_id = f_o[0]
    friends = f_o[1]
    connections = []
    for friend_id in friends:
        key = (user_id, friend_id)
        if user_id > friend_id:
            key = (friend_id, user_id)
        connections.append((key, 0))  # they are friends, value=0
    for friend_pair in itertools.combinations(friends, 2):
        friend_0 = friend_pair[0]
        friend_1 = friend_pair[1]
        key = (friend_0, friend_1)
        if friend_0 > friend_1:
            key = (friend_1, friend_0)
        connections.append((key, 1))  # they have mutual friends, value=1
    return connections
    
def mutual_friend_count_to_recommendation(f):
    pair = f[0]
    friend0 = pair[0]
    friend1 = pair[1]
    noMutFriends = f[1]
    rec0 = (friend0, (friend1, noMutFriends))
    rec1 = (friend1, (friend0, noMutFriends))
    return [rec0, rec1]

def recommendation_to_sorted_truncated(recs):
    recs.sort(key=lambda x: (-x[1], x[0]))
    return list(map(lambda x: x[0], recs))[:10]
     

<IPython.core.display.Javascript object>

In [ ]:
# Read from text file
lines = sc.textFile("/content/soc-LiveJournal1Adj.txt")

# Map each line to the form: (user_id, [friend_id_0, friend_id_1, ...])
friend_ownership = lines.map(line_to_friend_ownership).filter(lambda friend: '' != friend[1])#.filter(lambda friend: 1000> friend[0]) #take 1000 samples for testing

# Map friend ownerships to instances of ((user_id, friend_id), VALUE).
# VALUE = 0 => pairs are already friends.
# VALUE = 1 => pairs have mutual friends.
friend_edges = friend_ownership.flatMap(friend_ownership_to_connection)
friend_edges.cache()

# Filter pairs that are already friends
mutual_friend = friend_edges.groupByKey() \
    .filter(lambda edge: 0 not in edge[1]) \
    .flatMap(lambda x: [(x[0],item) for item in x[1]]) # flat it to count total mutual friends No; use map directly causes bugs

# Count mutual friends by adding up values
mutual_friend_counts = mutual_friend.reduceByKey( lambda x,y : x+y)

# Create the recommendation objects, group them by key, then sort and 
recommendations = mutual_friend_counts.flatMap(mutual_friend_count_to_recommendation).groupByKey() 

# Truncate the recommendations to the 10 most highly recommended.
recommendations10 = recommendations.map(lambda m: (m[0], recommendation_to_sorted_truncated(list(m[1])))).sortByKey() 

# Include in your writeup the recommendations for the users with following user IDs: 924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993.
results = recommendations10.filter(lambda recommendations: recommendations[0] in [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993])

<IPython.core.display.Javascript object>

In [ ]:
results.collect()

<IPython.core.display.Javascript object>

[(924, [439, 2409, 6995, 11860, 15416, 43748, 45881]),
 (8941, [8943, 8944, 8940]),
 (8942, [8939, 8940, 8943, 8944]),
 (9019, [9022, 317, 9023]),
 (9020, [9021, 9016, 9017, 9022, 317, 9023]),
 (9021, [9020, 9016, 9017, 9022, 317, 9023]),
 (9022, [9019, 9020, 9021, 317, 9016, 9017, 9023]),
 (9990, [13134, 13478, 13877, 34299, 34485, 34642, 37941]),
 (9992, [9987, 9989, 35667, 9991]),
 (9993, [9991, 13134, 13478, 13877, 34299, 34485, 34642, 37941])]

**Câu 2)** 


**Câu 2d)**

In [ ]:
from itertools import combinations
from collections import defaultdict

# Load dataset and remove duplicates
data = []
with open('/content/browsing.txt', 'r') as f:
    for line in f:
        items = set(line.strip().split())
        data.append(list(items))

In [ ]:
# First pass to find frequent items
min_support = 100
item_counts = defaultdict(int)
for basket in data:
    for item in basket:
        item_counts[item] += 1
frequent_items = set(item for item, count in item_counts.items() if count >= min_support)

In [ ]:
# Second pass to find frequent itemsets of size 2
itemset_counts = defaultdict(int)
for basket in data:
    for itemset in combinations(basket, 2):
        if set(itemset).issubset(frequent_items):
            itemset_counts[itemset] += 1
frequent_itemsets = set(itemset for itemset, count in itemset_counts.items() if count >= min_support)

In [ ]:
# Generate association rules for pairs of items
rules = []
for itemset in frequent_itemsets:
    for item in itemset:
        antecedent = frozenset([item])
        consequent = frozenset(frozenset(itemset) - antecedent)
        confidence = itemset_counts[itemset] / item_counts[item]
        rules.append((antecedent, consequent, confidence))

In [ ]:
# Sort rules by confidence and print top 5
top_rules = sorted(rules, key=lambda x: (-x[2], tuple(x[0])))
for antecedent, consequent, confidence in top_rules[:5]:
    print(f"{tuple(antecedent)} -> {tuple(consequent)} : {confidence}")

**Câu 2e)**

In [ ]:
from itertools import combinations
from collections import defaultdict

# Load dataset and remove duplicates
data = []
with open('/content/browsing.txt', 'r') as f:
    for line in f:
        items = set(line.strip().split())
        data.append(list(items))

In [ ]:
# First pass to find frequent items
min_support = 100
item_counts = defaultdict(int)
for basket in data:
    for item in basket:
        item_counts[item] += 1
frequent_items = set(item for item, count in item_counts.items() if count >= min_support)

In [ ]:
# Second pass to find frequent itemsets of size 3
itemset_counts = defaultdict(int)
for basket in data:
    for itemset in combinations(basket, 3):
        if set(itemset).issubset(frequent_items):
            itemset_counts[itemset] += 1
frequent_itemsets = set(itemset for itemset, count in itemset_counts.items() if count >= min_support)

In [ ]:
# Generate association rules for pairs of items
rules = []
for itemset in frequent_itemsets:
    for item in itemset:
        antecedent = frozenset([item])
        consequent = frozenset(frozenset(itemset) - antecedent)
        confidence = itemset_counts[itemset] / item_counts[item]
        rules.append((antecedent, consequent, confidence))

In [ ]:
# Sort rules by confidence and print top 5
top_rules = sorted(rules, key=lambda x: (-x[2], tuple(x[0])))
for antecedent, consequent, confidence in top_rules[:5]:
    print(f"{tuple(antecedent)} -> {tuple(consequent)} : {confidence}")

Câu 4

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import pdb
import unittest
from PIL import Image

# Finds the L1 distance between two vectors
# u and v are 1-dimensional np.array objects
# TODO: Implement this
def l1(u, v):
    return np.sum(np.abs(u-v))

# Loads the data into a np array, where each row corresponds to
# an image patch -- this step is sort of slow.
# Each row in the data is an image, and there are 400 columns.
def load_data(filename):
    return np.genfromtxt(filename, delimiter=',')

# Creates a hash function from a list of dimensions and thresholds.
def create_function(dimensions, thresholds):
    def f(v):
        boolarray = [v[dimensions[i]] >= thresholds[i] for i in range(len(dimensions))]
        return "".join(map(str, map(int, boolarray)))
    return f

# Creates the LSH functions (functions that compute L K-bit hash keys).
# Each function selects k dimensions (i.e. column indices of the image matrix)
# at random, and then chooses a random threshold for each dimension, between 0 and
# 255.  For any image, if its value on a given dimension is greater than or equal to
# the randomly chosen threshold, we set that bit to 1.  Each hash function returns
# a length-k bit string of the form "0101010001101001...", and the L hash functions 
# will produce L such bit strings for each image.
def create_functions(k, L, num_dimensions=400, min_threshold=0, max_threshold=255):
    functions = []
    for i in range(L):
        dimensions = np.random.randint(low = 0, 
                                   high = num_dimensions,
                                   size = k)
        thresholds = np.random.randint(low = min_threshold, 
                                   high = max_threshold + 1, 
                                   size = k)

        functions.append(create_function(dimensions, thresholds))
    return functions

# Hashes an individual vector (i.e. image).  This produces an array with L
# entries, where each entry is a string of k bits.
def hash_vector(functions, v):
    return np.array([f(v) for f in functions])

# Hashes the data in A, where each row is a datapoint, using the L
# functions in "functions."
def hash_data(functions, A):
    return np.array(list(map(lambda v: hash_vector(functions, v), A)))

# Retrieve all of the points that hash to one of the same buckets 
# as the query point.  Do not do any random sampling (unlike what the first
# part of this problem prescribes).
# Don't retrieve a point if it is the same point as the query point.
def get_candidates(hashed_A, hashed_point, query_index):
    return filter(lambda i: i != query_index and \
        any(hashed_point == hashed_A[i]), range(len(hashed_A)))

# Sets up the LSH.  You should try to call this function as few times as 
# possible, since it is expensive.
# A: The dataset in which each row is an image patch.
# Return the LSH functions and hashed data structure.
def lsh_setup(A, k = 24, L = 10):
    functions = create_functions(k = k, L = L)
    hashed_A = hash_data(functions, A)
    return (functions, hashed_A)

# Run the entire LSH algorithm
def lsh_search(A, hashed_A, functions, query_index, num_neighbors = 10):
    hashed_point = hash_vector(functions, A[query_index, :])
    candidate_row_nums = get_candidates(hashed_A, hashed_point, query_index)
    
    distances = map(lambda r: (r, l1(A[r], A[query_index])), candidate_row_nums)
    best_neighbors = sorted(distances, key=lambda t: t[1])[:num_neighbors]

    return [t[0] for t in best_neighbors]

# Plots images at the specified rows and saves them each to files.
def plot(A, row_nums, base_filename):
    for row_num in row_nums:
        patch = np.reshape(A[row_num, :], [20, 20])
        im = Image.fromarray(patch)
        if im.mode != 'RGB':
            im = im.convert('RGB')
        im.save(base_filename + "-" + str(row_num) + ".png")

# Finds the nearest neighbors to a given vector, using linear search.
def linear_search(A, query_index, num_neighbors):
    query_vec = A[query_index, :]
    search_results_list = []
    idx_list = filter(lambda x: x!=query_index, range(len(A)))
    for idx in idx_list:
        search_results_list.append((idx, l1(query_vec, A[idx, :])))
    best_neighbors = sorted(search_results_list, key=lambda x: x[1])[:num_neighbors]
    return [t[0] for t in best_neighbors]

# TODO: Write a function that computes the error measure
def compute_error_measure(A, hashed_A, functions, j_list, num_neighbors=3):
    error_sum = 0
    for j in j_list:
        lsh_top_neighbors = lsh_search(A, hashed_A, functions, j, num_neighbors)
        lin_top_neighbors = linear_search(A, j, num_neighbors)
        error_sum += np.sum([l1(A[j, :], A[top_n, :]) for top_n in lsh_top_neighbors]) / \
                     np.sum([l1(A[j, :], A[top_n, :]) for top_n in lin_top_neighbors])
    return error_sum / len(j_list)


# TODO: Solve Problem 4
def problem4(file_path):

    # Load data and setup LSH
    data = load_data(file_path)
    h_funcs_og, h_data_og = lsh_setup(data)

    # Average time for top 3 near neighbors
    query_indices = range(100, 1001, 100)

    time_vals = []
    for query_index in query_indices:
        t0 = time.time()
        lsh_search(data, h_data_og, h_funcs_og, query_index, 3)
        t1 = time.time()
        time_vals.append(t1-t0)
    print(f'Thời gian chạy trung bình của LSH: {np.mean(time_vals):.3f} giây')

    time_vals = []
    for query_index in query_indices:
        t0 = time.time()
        linear_search(data, query_index, 3)
        t1 = time.time()
        time_vals.append(t1-t0)
    print(f'Thời gian chạy trung bình của linear search {np.mean(time_vals):.3f} giây')

    # Plot the error value as a function of L:
    l_vals = range(10, 21, 2)
    e_vals = []
    for l_val in l_vals:
        h_funcs, h_data = lsh_setup(data, L=l_val, k=24)
        e_vals.append(compute_error_measure(data, h_data, h_funcs, query_indices, 3))

    fig, ax = plt.subplots()
    ax.plot(l_vals, e_vals)
    ax.set(xlabel='L', ylabel='Error Measure', title='Error Measure as a Function of L')
    fig.savefig("error_vs_L.png")

    # Plot the error value as a function of k:
    k_vals = range(16, 25, 2)
    e_vals = []
    for k_val in k_vals:
        h_funcs, h_data = lsh_setup(data, k=k_val, L=10)
        e_vals.append(compute_error_measure(data, h_data, h_funcs, query_indices, 3))

    fig, ax = plt.subplots()
    ax.plot(k_vals, e_vals)
    ax.set(xlabel='k', ylabel='Error Measure', title='Error Measure as a Function of k')
    fig.savefig("error_vs_k.png")

    # Plot top 10 neighbors found using both methods
    plot(data, [100], 'query_img_')
    plot(data, lsh_search(data, h_data_og, h_funcs_og, 100, 10), 'lsh_')
    plot(data, linear_search(data, 100, 10), 'lin_')

#### TESTS #####

class TestLSH(unittest.TestCase):
    def test_l1(self):
        u = np.array([1, 2, 3, 4])
        v = np.array([2, 3, 2, 3])
        self.assertEqual(l1(u, v), 4)

    def test_hash_data(self):
        f1 = lambda v: sum(v)
        f2 = lambda v: sum([x * x for x in v])
        A = np.array([[1, 2, 3], [4, 5, 6]])
        self.assertEqual(f1(A[0,:]), 6)
        self.assertEqual(f2(A[0,:]), 14)

        functions = [f1, f2]
        self.assertTrue(np.array_equal(hash_vector(functions, A[0, :]), np.array([6, 14])))
        self.assertTrue(np.array_equal(hash_data(functions, A), np.array([[6, 14], [15, 77]])))

    ### TODO: Write your tests here (they won't be graded, 
    def test_linear_search(self):
        q_ind = 0
        A = np.array([[0,0,0], [5,0,0], [6,0,0], [7,0,0], [7,5,5], [7,7,7]])
        self.assertEqual(linear_search(A, q_ind, 3), [1,2,3])

In [7]:
file_path = "/content/patches.csv"
problem4(file_path)

ValueError: ignored